# Adsorbate Fingerprints Setup

In this tutorial we will try the adsorbate fingerprint generator, which is useful for converting adsorbates on extended surfaces into fingerprints for predicting their chemisorption energies, bond lengths or other properties.

In [18]:
# Import packages.
import numpy as np
import ase.io
from ase.data import atomic_numbers
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from catlearn.fingerprint.setup import FeatureGenerator
from catlearn.fingerprint.periodic_table_data import get_radius, default_catlearn_radius
from catlearn.fingerprint.adsorbate_prep import autogen_info
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    plot = True
except ImportError:
    print('Pandas and seaborn modules are needed for this tutorial.')

First we use ASE to generate some adsorbate/surface systems from ASE.

We return the atoms objects in a list, which is the simplest format and easily transferable to CatLearn.

In [19]:
"""Make a list of atoms objects."""
adsorbates = ['H', 'O', 'C', 'N', 'S', 'Cl', 'P', 'F']
symbols = ['Ag', 'Au', 'Cu', 'Pt', 'Pd', 'Ir', 'Rh', 'Ni', 'Co']
images = []
for i, s in enumerate(symbols):
    rs = get_radius(atomic_numbers[s])
    a = 2 * rs * 2 ** 0.5
    for ads in adsorbates:
        atoms = fcc111(s, (2, 2, 3), a=a)
        atoms.center(vacuum=6, axis=2)
        c_atoms = [a.index for a in atoms if
                   a.z < atoms.cell[2, 2] / 2. + 0.1]
        atoms.set_constraint(FixAtoms(c_atoms))
        h = (default_catlearn_radius(
            atomic_numbers[ads]) + rs) / 2 ** 0.5
        add_adsorbate(atoms, ads, h, 'bridge')
        images.append(atoms)

Here we have our list of atoms stored in `images`.

The adsorbate fingerprinter generates fingerprint based on connectivity of atoms in the adsorbate/slab system. It therefore uses certain metadata as intermediates between the atoms object and the fingerprint. A connectivity matrix is one of those metadata which can some times be computationally time consuming to generate and therefore needs to be made only once.

A list of raw atoms without the metadata can be feed through `autogen_info` to attach the connectivity matrix and metadata.

In [20]:
images = autogen_info(images)

  0%|          | 0/72 [00:00<?, ?it/s]/Users/mhangaard/src/CatLearn/catlearn/fingerprint/adsorbate_prep.py:78: UserWarning: 'species' key missing.
  warnings.warn("'species' key missing.")
/Users/mhangaard/src/CatLearn/catlearn/fingerprint/adsorbate_prep.py:285: UserWarning: No adsorption site detected.
  warnings.warn(msg)
100%|██████████| 72/72 [00:00<00:00, 1827.78it/s]

[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []


In [ ]:

# Get the fingerprint generator.
fingerprint_generator = FeatureGenerator()

# List of functions to call.
feature_functions = [fingerprint_generator.mean_site,
                     fingerprint_generator.mean_surf_ligands]
# There are many more available.

# Generate the data
training_data = fingerprint_generator.return_vec(structures, feature_functions)

# Get a list of names of the features.
feature_names = fingerprint_generator.return_names(feature_functions)

for l in range(len(feature_names)):
    print(l, feature_names[l])

# Select some features to plot.
selection = [10, 11, 14]

# Plot selected of the feature distributions.
data = {}
traint = np.transpose(training_data[:, selection])
for i, j in zip(traint, selection):
    data[j] = i
df = pd.DataFrame(data)
fig = plt.figure(figsize=(20, 10))
ax = sns.violinplot(data=df, inner=None)
plt.title('Feature distributions')
plt.xlabel('Feature No.')
plt.ylabel('Distribution.')

string = 'Plotting:'
for s in selection:
    string += '\n' + feature_names[s]
print(string)

Attached to the atoms objects, the fingerprinter needs information about the atoms belonging to the adsorbate.
Either one can identify the atomic indices in `atoms.info['ads_index']`, or one can specify the chemical formula in `atoms.info['key_value_pairs']['species']`.

  ```python
    atoms.subsets['ads_atoms'] = dictionary[f]['ads_index']
    atoms.info['key_value_pairs']['species'] = 'CH3'
    structures.append(atoms)
  ```